In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from typing import Tuple, Union
from datasets import DF, Load, Save

In [2]:
df = Load(DF.WDI)

df = df[df['Indicator Code'] == 'NY.GDP.PCAP.CD']
df.drop(columns = ['Indicator Name', 'Indicator Code', 'Country Name'], inplace = True)

In [6]:
def compute(df: pd.DataFrame, y0: Union[str, int], y1: Union[str, int]) -> Tuple[pd.DataFrame, list[str]]:
    y0, y1 = str(y0), str(y1)

    df = df[['Country Code', y0, y1]]

    ctr_drop = df[df[[y0, y1]].isna().any(axis = 1)]['Country Code'].to_list()
    df = df.dropna(how = 'any')

    df = df.reset_index(drop = True)
    df = df.drop(columns = ['Country Code'])

    df = pd.DataFrame(np.sort(df.values, axis = 0), index = df.index, columns = df.columns)

    df['Growth'] = (df[y1] - df[y0]) / df[y0] * 100
    df['Rank'] = [str(i) for i in range(len(df), 0, -1)]

    return df, ctr_drop

In [9]:
col_curve = '#286bbb'
col_acc = '#883039'
col_acc_scnd = '#956a38'

fc = '#595959'
fc_scnd = '#8c8c8c'
fc_title = '#262626'

lw = 2


def plot(df: pd.DataFrame, ctr_d: list[str]) -> go.Figure:
    y0, y1 = df.columns[0], df.columns[1]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x = df['Rank'], y = df['Growth'], line = dict(width = lw), showlegend = False,
            mode = 'markers+lines+text', marker=dict(color=col_curve, size=3.5)
    ))
    
    fig.update_layout(
        title = dict(text = f'Growth Incidence Curve ({y0}-{y1})', font = dict(size=20), 
                     xanchor = 'center', yanchor = 'middle'),
        template = 'seaborn', width = 1000, height = 550,
        font_family = "Helvetica Now Text", font_color = fc, title_font_color = fc_title,
        margin = dict(t=50, b=50, l=10, r=25), 
        plot_bgcolor = 'rgba(234, 234, 243, 0.65)',
        yaxis = dict(ticksuffix = "%  "),
        xaxis = dict(
            tickmode="array",
            tickvals=df['Rank'],
            ticktext=df['Rank'],
            range = [-0.5, len(df['Rank']) - 0.5]
        ),
        legend=dict(
            orientation='h', y = 1, xanchor='center', x = 0.9
        )
    )

    # mean_of_gr = df['Growth'].mean()
    gr_in_mean = (df[y1].mean() - df[y0].mean()) / df[y0].mean() * 100

    # fig.add_hline(y = mean_of_gr, name = 'Mean of Growth Rate', showlegend = True, line = dict(color = col_acc, width = lw, dash = 'longdash'), opacity = 0.8)
    fig.add_hline(y = gr_in_mean, name = 'Growth Rate in Mean', showlegend = False, line = dict(color = col_acc, width = lw), opacity = 0.8)
    fig.add_annotation(
        text = f'Growth in Mean: {round(gr_in_mean, 2)}%', showarrow = False, x = 1.2, y = gr_in_mean + 1.75,
        xanchor='left', yanchor = 'bottom', font = dict(size = 12, color = fc)
    )

    fig.add_annotation(
        text = 'Source: World Bank National Accounts data (2024).', showarrow = False, x = 0, y = -0.1,
        xref = 'paper', yref ='paper', xanchor = 'left', yanchor = 'bottom',
        font = dict(size = 10, color = fc_scnd), align = 'left'
    )

    if ctr_d:
        
        if len(ctr_d) > 1:
            ctr_fmtd = ', '.join(ctr_d[:-1]) + ' and ' + ctr_d[-1]
        else:
            ctr_fmtd = ctr_d[0]

        fig.add_annotation(
            text = f'Note: {ctr_fmtd} have been excluded due to data unavailability.', showarrow = False, x = 1, y = -0.1,
            xref = 'paper', yref ='paper', xanchor = 'right', yanchor = 'bottom',
            font = dict(size = 10, color = fc_scnd), align = 'left'
        )

    return fig

### Analysis: 1980 - 2000

In [7]:
g_fst, d_fst = compute(df, 1980, 2000)
print('Countries dropped:', *d_fst)

Countries dropped: AFG ARG BRA COD IMN LAO LBN MOZ NIC PER VNM YEM


In [10]:
f_fst = plot(g_fst, d_fst)
f_fst.show()

### Analysis: 2000 - 2020

In [11]:
g_snd, d_snd = compute(df, 2000, 2020)
print('Countries dropped:', *d_snd)

Countries dropped:


In [12]:
f_snd = plot(g_snd, d_snd)
f_snd.show()

### Analysis: 1980 - 2020

In [13]:
g_all, d_all = compute(df, 1980, 2020)
print('Countries dropped:', *d_all)

Countries dropped: AFG ARG BRA COD IMN LAO LBN MOZ NIC PER VNM YEM


In [14]:
f_all = plot(g_all, d_all)
f_all.show()

In [15]:
dir_path = '../images/'

f_fst.write_image(dir_path + 'gic-1980-2000.png', scale=3, engine='kaleido')
f_snd.write_image(dir_path + 'gic-2000-2020.png', scale=3, engine='kaleido')
f_all.write_image(dir_path + 'gic-1980-2020.png', scale=3, engine='kaleido')